In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

from classifier import GolemClassifier
from dataloaders import get_dataloaders
from model_utils import train_epoch, eval

In [3]:
train_loader, test_loader, classes = get_dataloaders()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18()
gc = GolemClassifier(resnet18, resnet18.fc.out_features)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gc.parameters(), lr=1e-3)

In [5]:
for i in range(2):
    loss = train_epoch(gc, criterion, optimizer, train_loader, device)
    print(f"{i+1}/2: loss={loss}")

1/2: loss=2353.923464536667
2/2: loss=1674.9488796591759


In [6]:
print(gc)

GolemClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra